# C07_CNN(Convolution Neural Network: 합성신경망)


#### CNN(Convolutional Neural Network)은 이미지의 공간 정보를 유지하면서 인접 이미지와의 특징을 효과적으로
#### 인식하고 강조하는 방식으로 이미지의 특징을 추출하는 부분과 이미지를 분류하는 부분으로 구성된다. 
#### 특징 추출 영역은 Filter를 사용하여 공유 파라미터 수를 최소화하면서 이미지의 특징을 찾는 Convolution 레이어와
#### 특징을 강화하고 모으는 Pooling 레이어로 구성된다.
#### CNN은 Filter의 크기, Stride, Padding과 Pooling 크기로 출력 데이터 크기를 조절하고, 
#### 필터의 개수로 출력 데이터의 채널을 결정한다.
#### CNN는 같은 레이어 크기의 Fully Connected Neural Network와 비교해 볼 때, 
#### 학습 파라미터양은 20% 규모다.
#### 은닉층이 깊어질 수록 학습 파라미터의 차이는 더 벌어진다.
#### CNN은 Fully Connected Neural Network와 비교하여 더 작은 학습 파라미터로 더 높은 인식률을 제공한다.

In [2]:
import tensorflow as tf

In [6]:
#Tensorflow에 기본적으로 내장되어 있는 데이터를 로드
from tensorflow.examples.tutorials.mnist import input_data
# 해당 폴더에 내용이 없으면,자동으로 데이터를 다운받기
mnist=input_data.read_data_sets('C:/JupyterSpace/datasets/mnist/data',one_hot=True)

Extracting C:/JupyterSpace/datasets/mnist/data\train-images-idx3-ubyte.gz
Extracting C:/JupyterSpace/datasets/mnist/data\train-labels-idx1-ubyte.gz
Extracting C:/JupyterSpace/datasets/mnist/data\t10k-images-idx3-ubyte.gz
Extracting C:/JupyterSpace/datasets/mnist/data\t10k-labels-idx1-ubyte.gz


In [34]:
#이미지(28X28) 갯수가 784:X
X=tf.placeholder(tf.float32, [None,28,28, 1])
#0~9 분류:Y
Y=tf.placeholder(tf.float32, [None, 10])

keep_prob=tf.placeholder(tf.float32)

In [35]:
#hidde layer0
W1=tf.Variable(tf.random_normal([3, 3, 1, 32],stddev=0.01))
#입력값에 가중치를 곱하고 relu함수를 이용해서 레이어 만들기
#padding='SAME': 이미지의 가장 외곽밖에서 한칸 밖으로 움직이는 옵션(Padding)
L1=tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1=tf.nn.relu(L1)
# Polling
L1=tf.nn.max_pool(L1,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [36]:
# dropout: overfitting방지위한 것
#L1=tf.nn.dropout(L1, keep_prob)
#
W2=tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2=tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [43]:
W3=tf.Variable(tf.random_normal([7*7*64,256], stddev=0.01))
L3=tf.reshape(L2, [-1,7*7*64])
L3=tf.matmul(L3, W3)
L3=tf.nn.relu(L3)
L3=tf.nn.dropout(L3, keep_prob)

In [44]:
#최종 결과값L3 출력256개를 입력값으로 받아서 0~9 출력값 만들기
W4=tf.Variable(tf.random_normal([256,10], stddev=0.01))
model=tf.matmul(L3, W4)

In [45]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(0.001).minimize(cost)

In [46]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

In [47]:
batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)

In [48]:
#epoch:학습 데이터를 전체를 한번 수행하는 것
for epoch in range(15):
    total_cost=0
    
    for i in range(total_batch):
        #지정한 크기만큼 학습데이터를 가져오기
        batch_xs, batch_ys =mnist.train.next_batch(batch_size)
        
        #추가 : 이미지 데이터를 cnn model 형태로 재구성 ([28,28,1])
        batch_xs=batch_xs.reshape(-1,28,28,1)
        _, cost_val=sess.run([optimizer, cost],feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
        total_cost += cost_val
        
    print("Epoch:", '%04d' % (epoch+1), 'Avg.Cost= ','{:.3f}'.format(total_cost/total_batch))
print('최적화 완료')

Epoch: 0001 Avg.Cost=  0.342
Epoch: 0002 Avg.Cost=  0.105
Epoch: 0003 Avg.Cost=  0.074
Epoch: 0004 Avg.Cost=  0.059
Epoch: 0005 Avg.Cost=  0.046
Epoch: 0006 Avg.Cost=  0.041
Epoch: 0007 Avg.Cost=  0.035
Epoch: 0008 Avg.Cost=  0.033
Epoch: 0009 Avg.Cost=  0.028
Epoch: 0010 Avg.Cost=  0.025
Epoch: 0011 Avg.Cost=  0.021
Epoch: 0012 Avg.Cost=  0.020
Epoch: 0013 Avg.Cost=  0.017
Epoch: 0014 Avg.Cost=  0.017
Epoch: 0015 Avg.Cost=  0.015
최적화 완료


In [49]:
#(tf.argmax(model, 1) : model의 행렬을 : 열 순서로 최대값 구하기
is_correct=tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
# cast(): 0과 1 반환
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:',sess.run(accuracy, feed_dict={X:mnist.test.images.reshape(-1,28,28,1), 
                                           Y:mnist.test.labels, keep_prob:1}))

정확도: 0.9882
